# Scraper Checks

In [69]:
import sqlite3
conn = sqlite3.connect('../../reddit-engagement-files/activity.db')
cursor = conn.cursor()
cursor.execute('''SELECT * FROM activity;''')
results = cursor.fetchall()
print(len(results))

1453


# NLP For Binary Classification

In [70]:
import pandas as pd
df = pd.read_sql_query('SELECT * FROM activity;', conn)
print(len(df))
df.head()

1453


,id,title,subreddit,access_time,action
0,l9c06q,Avergage people are just as greedy as rich peo...,unpopularopinion,01/31/2021 20:39:56,down
1,l9bayq,Obese people infuriate me.,TrueOffMyChest,01/31/2021 20:39:56,down
2,l90oq6,I am proud to do my part in paying forward our...,wallstreetbets,01/31/2021 20:39:56,down
3,l8xsfa,"Cool, but why would they bother telling us?",wallstreetbets,01/31/2021 20:39:56,down
4,kzul59,There is absolutely no goddamn reason why you ...,unpopularopinion,01/31/2021 20:39:56,down


## Preprocessing

In [71]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import re

In [65]:
def preprocess(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'http\S+', '', text) # remove urls
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    # remove emojis
    # remove stop words
    # remove special characters
    # expand contractions
    # tokenize
    
    print(text)

In [68]:
preprocess('9cat0cat')

catcat


In [ ]:

    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)

    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)